<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine/CNNmodelv2_v2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Builf train and evaluate
# Save model for offline scoring on test
# Model name: CCP_ICD10_Searchengine/CNNmodel_v1.0.ipynb
# References
#https://github.com/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments/Model-2-keras_lstmConv.ipynb
#https://github.com/njainds/NLP/blob/master/kaggle/QIQC/22nd%20Place%20Solution%20-%206%20models%20%2B%20POS%20tagging.ipynb
#https://github.com/Cheneng/DPCNN/blob/master/model/DPCNN.py


In [2]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:

import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')

import os
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam, RMSprop
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import callbacks

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from gensim.models.keyedvectors import KeyedVectors

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2"
os.chdir("/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
embedding_matrix.npy  model_data.csv  tokenizer.pickle	word_index.npy
icd_dict.npy	      test_X.npy      train_X.npy
mispell_dict.npy      test_y.npy      train_y.npy


In [0]:
x_train = np.load("train_X.npy")
x_test = np.load("test_X.npy")
y_train = np.load("train_y.npy")
y_test = np.load("test_y.npy")
features = np.load("test_y.npy")
icd_dict = np.load("icd_dict.npy").item()
word_index = np.load("word_index.npy").item()
embeddings = np.load("embedding_matrix.npy")

In [0]:
class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=4590., mode='triangular',gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
    
        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size,
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
class RocAucEvaluation(keras.callbacks.Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

del check_point, ra_val, early_stop
!rm ./best_model.hdf5
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(x_test, y_test), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

In [0]:
#configs
max_features = embeddings.shape[0]
maxlen = x_train.shape[1]
embed_size = embeddings.shape[1]
n_class = y_train.shape[1]

In [0]:
def get_model():
    del model
    inp = Input(shape=(maxlen,))
    x   = Embedding(max_features, embed_size, weights=[embeddings], trainable=True)(inp)
    x   = SpatialDropout1D(rate=0.2)(x)
    x1  = Conv1D(128, kernel_size=1, padding='valid', kernel_initializer='he_uniform')(x)
    x2  = Conv1D(128, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x)
    x3  = Conv1D(128, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x)
    x4  = Conv1D(128, kernel_size=4, padding='valid', kernel_initializer='he_uniform')(x)
    x5  = Conv1D(128, kernel_size=5, padding='valid', kernel_initializer='he_uniform')(x)
    
    x1  = GlobalMaxPooling1D()(x1)
    x2  = GlobalMaxPooling1D()(x2)
    x3  = GlobalMaxPooling1D()(x3)
    x4  = GlobalMaxPooling1D()(x4)
    x5  = GlobalMaxPooling1D()(x5)
    c   = concatenate([x1,x2,x3,x4,x5])
    y   = Dense(512, activation='relu')(c)
    y   = Dropout(0.2)(y)
    out = Dense(n_class, activation='softmax')(y)
    model=Model(inputs=inp,outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=["accuracy"])
    return model
#print(model.summary())

In [178]:
x_train.shape

(54151, 7)

In [0]:
# Kfold training of model

kfold = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=True)
y_test = np.zeros((x_test.shape[0], n_class))

for i, (train_index, valid_index) in enumerate(kfold.split(x_train, np.argmax(y_train, axis=1))):
    X_train, X_val, Y_train, Y_val = x_train[train_index], x_train[valid_index], y_train[train_index], y_train[valid_index]
    
    file_path = "best_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
    ra_val = RocAucEvaluation(validation_data=(X_val, Y_val), interval = 1)
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
    
    model = get_model()
    
    print(\"### Model: {} with seed: {}  for fold no. {} ####\".format(emb, seed, i))
    
    clr = CyclicLR(base_lr=1e-6, max_lr=5e-4,step_size=11281., mode='exp_range',gamma=0.99994)
    model.fit(X_train, Y_train, batch_size = 24, epochs = 10, validation_data = (X_val, Y_val), verbose = 1, callbacks = [clr, ra_val, check_point, early_stop])
    
    y_pred = model.predict([X_val], batch_size=batch_size, verbose=2)
    y_test += model.predict([x_test], batch_size=16, verbose=1)/5
    os.remove(file_path)
return y_test


In [169]:
model.fit(x_train, y_train, batch_size = 24, epochs = 10, validation_data = (x_test, y_test), verbose = 1, callbacks = [ra_val, check_point, early_stop])


Train on 54151 samples, validate on 6017 samples
Epoch 1/10
54151/54151 [==============================] - 35s 646us/step - loss: 3.8050 - acc: 0.2944 - val_loss: 2.5542 - val_acc: 0.4742

 ROC-AUC - epoch: 1 - score: 0.956673

Epoch 00001: val_loss improved from inf to 2.55424, saving model to best_model.hdf5
Epoch 2/10
54151/54151 [==============================] - 33s 603us/step - loss: 2.3114 - acc: 0.5035 - val_loss: 1.9375 - val_acc: 0.5828

 ROC-AUC - epoch: 2 - score: 0.973830

Epoch 00002: val_loss improved from 2.55424 to 1.93754, saving model to best_model.hdf5
Epoch 3/10
54151/54151 [==============================] - 32s 594us/step - loss: 1.8721 - acc: 0.5830 - val_loss: 1.7013 - val_acc: 0.6266

 ROC-AUC - epoch: 3 - score: 0.978860

Epoch 00003: val_loss improved from 1.93754 to 1.70131, saving model to best_model.hdf5
Epoch 4/10
54151/54151 [==============================] - 33s 603us/step - loss: 1.6411 - acc: 0.6282 - val_loss: 1.5780 - val_acc: 0.6540

 ROC-AUC - epo

In [76]:
# ROC scores on Test set for different codes

y_pred = model.predict(x_test, batch_size = 16, verbose = 1)
print(y_pred[:,395].shape)
scores=[]
for i in range(396):
  score = roc_auc_score(y_test[:,i], y_pred[:,i])
  scores.append(score)
  #print("score for index %s is %d" (i,score))

low = list(np.argsort(scores)[:5])
high = list(np.argsort(scores)[::-1][:250])
print('high scores is {}'.format(np.mean([scores[i] for i in high])))
print('low scores is {}'.format(np.mean([scores[i] for i in low])))
# ROC is not a problem in multi-class prediction

6017/6017 [==============================] - 1s 177us/step
(6017,)
high scores is 0.9967437630437778
low scores is 0.8340337086591365


In [170]:
# Top Codes with low accuracy on test set
test_pred = np.argmax(model.predict(x_test, batch_size = 16, verbose = 1), axis=1)
test_act = np.argmax(y_test, axis=1) 
mdf = pd.DataFrame({'test_act':list(test_act),'test_pred':list(test_pred)})
acc = [len(mdf[(mdf['test_pred']==i) & (mdf['test_act']==i)])/len(mdf[mdf['test_act']==i]) for i in mdf['test_act'].values]
mdf2 = pd.DataFrame({'class_index': mdf['test_act'].values,'accuracy':acc})
print(len(mdf2[mdf2['accuracy']<0.5].class_index.unique().tolist()))
codes= mdf2[mdf2['accuracy']<0.5].class_index.unique().tolist()
#codes

6017/6017 [==============================] - 1s 206us/step
135


In [173]:
# Accuracy on various condifence levels and sample coverage on Test set
p = model.predict(x_test, batch_size = 16, verbose = 1)
test_pred = np.argmax(p, axis=1)
test_pred_proba = np.max(p,axis=1)
mdf3 = pd.DataFrame({'test_act':list(test_act),'test_pred':list(test_pred), 'prob': list(test_pred_proba)})
CUTOFF=0.8
acc = len(mdf3[(mdf3['prob']>CUTOFF) & (mdf3['test_act']==mdf3['test_pred'])])/len(mdf3[(mdf3['prob']>CUTOFF)])
print("Accuracy on Test dataset is {}".format(acc))
print("coverage with this accuracy level on Test dataset is {}".format(len(mdf3[(mdf3['prob']>CUTOFF)])/len(mdf3)))


6017/6017 [==============================] - 1s 211us/step
Accuracy on Test dataset is 0.9067292644757433
coverage with this accuracy level on Test dataset is 0.530995512713977


In [175]:
# Steps- for re generating the Data for modeling
#removing or balancing some ICDs

#datasize for icd codes
#strain = np.argmax(y_train, axis=1)
#codesize = dict([(i,sum(strain==i)) for i in codes]) 
#codesize


0

In [0]:
# Final results for presentation

#itoicd = dict((v,k) for k,v in icd_dict.items())
test_pred = np.argmax(model.predict(x_test, batch_size = 16, verbose = 1), axis=1)
test_act = np.argmax(y_test, axis=1)
trn_pred = np.argmax(model.predict(x_train, batch_size = 16, verbose = 1), axis=1)
trn_act = np.argmax(y_train, axis=1)

print("Accuracy on val dataset is {}".format(sum(test_act==test_pred)/test_pred.shape[0]))
print("Accuracy on trn dataset is {}".format(sum(trn_act==trn_pred)/trn_pred.shape[0]))

7834/7834 [==============================] - 1s 117us/step
Accuracy on val dataset is 0.7669345579793341
Accuracy on trn dataset is 0.857161092672964


In [0]:
# Saving Model
# serialize model to JSON
model_json = model.to_json()
with open("model_v1.0.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_v1.0.h5")
print("Saved model to disk")
 
# later...

Saved model to disk


In [0]:
# load json and create model
json_file = open('model_v1.0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_v1.0.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=["accuracy"])
score = np.argmax(loaded_model.predict(DATASET, batch_size = 16, verbose = 1), axis=1)